# Resources

Based om Module 2 of the Deep Learning course below (LangGraph)

* [A simple Python implementation of the ReAct pattern for LLMs](https://arc.net/l/quote/duflzttq)
  * Simon Willison Blog Article
* [Deep Learning AI Course, AI Agents with LangGraph](https://learn.deeplearning.ai/courses/ai-agents-in-langgraph/lesson/1/introduction)

### Assumptions

* Pythons setup (I created a 3.11 venv)
* OpenAI Key (Granite struggled but perhaps not a fair comparions ollama/grantite q4 8b v `gpt-4o`

In [1]:
import openai
import re
import httpx
import os
import rich
import json
from openai import OpenAI
from agents import Agent, ModelSettings, function_tool,Runner
from rich.pretty import pprint


In [2]:
# Boilerplate for swapping in Granite via ollama
#model = "granite3-dense:8b"
#model = "granite3.1-dense:2b"
#client = OpenAI(
#     base_url='http://localhost:11434/v1',
#     api_key='ollama',
# ) 

from dotenv import load_dotenv
load_dotenv()

model = "gpt-4o"
client = OpenAI() 

In [3]:
# Quick test code - verify LLM conenctivity etc (disable via Raw)

chat_completion = client.chat.completions.create(
    model=model,
    messages=[{"role": "user", "content": "Write a simple Python example class called User"}],
    temperature=0,
)
print(model)
print(f"{chat_completion.choices[0].message.content}")

gpt-4o
Certainly! Below is a simple example of a Python class called `User`. This class includes basic attributes like `username` and `email`, and a method to display user information.

```python
class User:
    def __init__(self, username, email):
        self.username = username
        self.email = email

    def display_user_info(self):
        print(f"Username: {self.username}")
        print(f"Email: {self.email}")

# Example usage:
if __name__ == "__main__":
    user1 = User("john_doe", "john@example.com")
    user1.display_user_info()
```

### Explanation:
- **`__init__` Method**: This is the constructor method that initializes a new instance of the `User` class with a `username` and `email`.
- **Attributes**: `username` and `email` are instance attributes that store the user's information.
- **`display_user_info` Method**: This method prints the user's information to the console.
- **Example Usage**: The `if __name__ == "__main__":` block is used to demonstrate how to create a

In [4]:
class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = client.chat.completions.create(
            #model="gpt-4o",
            model=model,
            temperature=0,
            messages=self.messages)
        return completion.choices[0].message.content

# ReAct Agent Prompt

In [5]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run through one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

get_provision_status:
e.g. get_provision_status: guid
returns the status of a cloud deployment such as a virtual machine when gived a guid (globally unique identifier)

log_error:
e.g log_error: status
When a guid has a provision_status ERROR call this with the return value of get_provision_status

log_status:
e.g log_status: status
When a guid does not have an ERROR status call this with the return value of get_provision_status

Example session:

Question: What is the staus of cloud deployment with guid <guid>
Thought: I should look up the status with get_provision_status 
Action: get_provision_status: guid 
PAUSE:

You will be called again with this:

Observation: Guid status "SUCCESS: Completed"

You then call any necessary logging tools before outputing the status:

Answer: Guid status "SUCCESS: Completed"
""".strip()

In [6]:
import random

'''
First of the *fake* functions to test if the LLM/Prompt will ReAct correctly
taking different paths on different results
'''

def get_provision_status(guid):

    #// call to MCP Server AAP2 Controller
    # // foo = bar()
    
    status_messages = [
        "INFO: Initializing",
        "INFO: In progress",
        "ERROR: Failed",
        "ERROR: API Timeout",
        "ERROR: Rate Limited",
        "WARNING: Minor errors",
        "SUCCESS: Completed"
    ]
        # "INFO: Finalizing",
    # return random.choice(f"{guid} status: {status_messages}")
    status = random.choice(status_messages)
    return f"{guid} status: {status}"
    

In [7]:
def log_error(status):
    print(f"{status} Logged stateus to Slack.")
    print(f"{status} Opened Jira Ticket with Status.")
    return 0

def log_status(status):
    print(f"{status} Logged status to Slack.")
    return 0

In [8]:
known_actions = {
   "log_status": log_status,
   "log_error": log_error,
   "get_provision_status": get_provision_status,
}

In [9]:
abot = Agent(prompt)

In [10]:
for i, message in  enumerate(abot.messages):
    if message["role"] != "assistant":
        print(f"Step {i}: App -> LLM:\n")
    else:
        print(f"Step {i}: App <- LLM:\n")
    print(f"Role: {message['role']}\nContent:\n\n{message['content']}\n\n\n")

Step 0: App -> LLM:

Role: system
Content:

You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run through one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

get_provision_status:
e.g. get_provision_status: guid
returns the status of a cloud deployment such as a virtual machine when gived a guid (globally unique identifier)

log_error:
e.g log_error: status
When a guid has a provision_status ERROR call this with the return value of get_provision_status

log_status:
e.g log_status: status
When a guid does not have an ERROR status call this with the return value of get_provision_status

Example session:

Question: What is the staus of cloud deployment with guid <guid>
Thought: I should look up the status with get_provision_status 
Action: get_provision_sta

In [11]:
abot = Agent(prompt)
abot.messages

[{'role': 'system',
  'content': 'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run through one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\nget_provision_status:\ne.g. get_provision_status: guid\nreturns the status of a cloud deployment such as a virtual machine when gived a guid (globally unique identifier)\n\nlog_error:\ne.g log_error: status\nWhen a guid has a provision_status ERROR call this with the return value of get_provision_status\n\nlog_status:\ne.g log_status: status\nWhen a guid does not have an ERROR status call this with the return value of get_provision_status\n\nExample session:\n\nQuestion: What is the staus of cloud deployment with guid <guid>\nThought: I should look up the status with get_provision_status \nAction: get

In [12]:
for i, message in  enumerate(abot.messages):
    if message["role"] != "assistant":
        print(f"Step {i}: App -> LLM:\n")
    else:
        print(f"Step {i}: App <- LLM:\n")
    print(f"Role: {message['role']}\nContent:\n\n{message['content']}\n\n\n")

Step 0: App -> LLM:

Role: system
Content:

You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run through one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

get_provision_status:
e.g. get_provision_status: guid
returns the status of a cloud deployment such as a virtual machine when gived a guid (globally unique identifier)

log_error:
e.g log_error: status
When a guid has a provision_status ERROR call this with the return value of get_provision_status

log_status:
e.g log_status: status
When a guid does not have an ERROR status call this with the return value of get_provision_status

Example session:

Question: What is the staus of cloud deployment with guid <guid>
Thought: I should look up the status with get_provision_status 
Action: get_provision_sta

# So far a bit to much "Human in the Loop"

Let's automate all this

In [13]:
action_re = re.compile("^Action: (\w+): (.*)$")

<>:1: SyntaxWarning: invalid escape sequence '\w'
<>:1: SyntaxWarning: invalid escape sequence '\w'
/var/folders/61/bmwhyqss6rjdhl37b33pbz9m0000gn/T/ipykernel_82932/2177360788.py:1: SyntaxWarning: invalid escape sequence '\w'
  action_re = re.compile("^Action: (\w+): (.*)$")


In [14]:
def query(question, max_turns=5):
    i = 0
    bot = Agent(prompt)
    next_prompt = question
    print("Step 0")
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [
            action_re.match(a)
            for a in result.split('\n')
            if action_re.match(a)
        ]
        if actions:
            print(f"\nStep {i}")
            # print(f"Actions:\n\n{actions}")
            action, action_inputs = actions[0].groups()
            if action not in known_actions:
                raise Exception(f"Unknown action: {action}: -- running {action} {action_inputs}")
            observation = known_actions[action](action_inputs)
            print(f"Observation: {observation}")
            next_prompt = f"Observation: {observation}"
        else:
            return
        

In [15]:
question = """
I have a deployments running with guid: 1adr4 what is its status
"""

query(question)

Step 0
Thought: I should look up the status of the deployment with the provided guid using get_provision_status.
Action: get_provision_status: 1adr4
PAUSE.

Step 1
Observation: 1adr4 status: SUCCESS: Completed
log_status: 1adr4 status: SUCCESS: Completed


In [16]:

question = """
I have deployments running with guids: 1adr4, aabf5, 45663, and 45ghb
First get each provision status and log to any services that need to know
Once finished with all deployments output their guids, status, and logging services:

* In a simple table
* As JSON 
"""

query(question, max_turns=10)

Step 0
Thought: I will start by checking the provision status of each deployment using their respective guids. I will then log the status accordingly and finally output the results in both a table and JSON format.

Action: get_provision_status: 1adr4
PAUSE

Step 1
Observation: 1adr4 status: ERROR: Failed
Action: log_error: ERROR: Failed
PAUSE

Step 2
ERROR: Failed Logged stateus to Slack.
ERROR: Failed Opened Jira Ticket with Status.
Observation: 0
Thought: I have logged the error for the guid 1adr4. Now, I will proceed to check the provision status for the next guid, aabf5.

Action: get_provision_status: aabf5
PAUSE

Step 3
Observation: aabf5 status: ERROR: Rate Limited
Action: log_error: ERROR: Rate Limited
PAUSE

Step 4
ERROR: Rate Limited Logged stateus to Slack.
ERROR: Rate Limited Opened Jira Ticket with Status.
Observation: 0
Thought: I have logged the error for the guid aabf5. Now, I will proceed to check the provision status for the next guid, 45663.

Action: get_provision_sta

# Function Calling
- Does not work with model = *granite*

In [17]:
import requests

def get_weather(latitude, longitude):
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m")
    data = response.json()
    return data['current']['temperature_2m']

In [18]:
tools = [{
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "Get current temperature for provided coordinates in celsius.",
        "parameters": {
            "type": "object",
            "properties": {
                "latitude": {"type": "number"},
                "longitude": {"type": "number"}
            },
            "required": ["latitude", "longitude"],
            "additionalProperties": False
        },
        "strict": True
    }
}]

messages = [{"role": "user", "content": "What's the weather like in Paris today?"}]

completion = client.chat.completions.create(
    model=model,
    messages=messages,
    tools=tools,
)

print(completion.choices[0].message.tool_calls)

[ChatCompletionMessageToolCall(id='call_lj8RnoGio7MXc4kOhQbF8Zdv', function=Function(arguments='{"latitude":48.8566,"longitude":2.3522}', name='get_weather'), type='function')]


In [19]:
tool_call = completion.choices[0].message.tool_calls[0]
args = json.loads(tool_call.function.arguments)

result = get_weather(args["latitude"], args["longitude"])
#print(result)

In [20]:
messages.append(completion.choices[0].message)  # append model's function call message
messages.append({                               # append result message
    "role": "tool",
    "tool_call_id": tool_call.id,
    "content": str(result)
})

completion_2 = client.chat.completions.create(
    model=model,
    messages=messages,
    tools=tools,
)
print(completion_2.choices[0].message.content)

The current temperature in Paris is 21.3°C.


# Reasoning
## Question: How many other Agent SDKs have this reasoning_effort or something similar exposed?
Is this worth talking about ?

In [21]:
model = "o3-mini"
prompt = """
Write a bash script that takes a matrix represented as a string with 
format '[1,2],[3,4],[5,6]' and prints the transpose in the same format.
"""

response = client.chat.completions.create(
    model = model,
    reasoning_effort="medium",
    messages=[
        {
            "role": "user", 
            "content": prompt
        }
    ]
)

pprint(response)
print('-----------------')
print(response.choices[0].message.content)

ChatCompletion(
│   id='chatcmpl-BIIVTAe49NPLZFzUQ2rD35Ilp9t9r',
│   choices=[
│   │   Choice(
│   │   │   finish_reason='stop',
│   │   │   index=0,
│   │   │   logprobs=None,
│   │   │   message=ChatCompletionMessage(
│   │   │   │   content='Below is one example of a Bash script that reads a matrix in the form\n\u2003\u2003[1,2],[3,4],[5,6]\nand prints its transpose in the same format. Save the script (for example, as transpose.sh),\nmake it executable (chmod +x transpose.sh), and run it by passing the matrix string as an argument.\n\n#!/bin/bash\n# Check if an argument is passed\nif [ $# -lt 1 ]; then\n  echo "Usage: $0 \'[1,2],[3,4],[5,6]\'"\n  exit 1\nfi\n\n# The input matrix string (for example: "[1,2],[3,4],[5,6]")\nmatrix_string="$1"\n\n# Extract rows using grep so that each row is of the form "[ … ]"\nrows=($(echo "$matrix_string" | grep -o \'\\[[^]]*\\]\'))\nnum_rows=${#rows[@]}\n\nif [ $num_rows -eq 0 ]; then\n  echo "Error: no rows found. Make sure your input is in the format \'[1,2],[3,4],[5,6]\'."\n  exit 1\nfi\n\ndeclare -a matrix\nnum_cols=0\n\n# Process each row to extract the individual numbers.\nfor (( i=0; i<num_rows; i++ )); do\n    # Remove the leading [ and trailing ]\n    row=$(echo "${rows[$i]}" | sed \'s/^\\[//; s/\\]$//\')\n    # Split the row on commas into an array\n    IFS=\',\' read -r -a elements <<< "$row"\n\n    # Set the number of columns based on the first row\n    if [ $i -eq 0 ]; then\n        num_cols=${#elements[@]}\n    fi\n\n    # Save the elements into a flattened matrix (row-major order)\n    for (( j=0; j<num_cols; j++ )); do\n        matrix[$(( i * num_cols + j ))]=${elements[$j]}\n    done\ndone\n\n# Build the transpose.\n# The transpose will have "num_cols" rows and "num_rows" columns.\nresult=""\nfor (( j=0; j<num_cols; j++ )); do\n    row_trans="["\n    for (( i=0; i<num_rows; i++ )); do\n         element=${matrix[$(( i * num_cols + j ))]}\n         row_trans+="$element"\n         # Add a comma unless this is the last element in the row.\n         if [ $i -ne $(( num_rows - 1 )) ]; then\n             row_trans+=","\n         fi\n    done\n    row_trans+="]"\n    # Add a comma between rows (except after the last one)\n    if [ $j -ne $(( num_cols - 1 )) ]; then\n         result+="$row_trans,"\n    else\n         result+="$row_trans"\n    fi\ndone\n\n# Print the transposed matrix\necho "$result"\n\n--------------------------------------------------\n\nHow the script works:\n\n1. It first ensures an argument (the matrix string) is provided.\n2. It uses grep to extract each row (pattern "\\[[^]]*\\]") so that each match is something like "[1,2]".\n3. It then removes the brackets and splits each row into its elements.\n4. The matrix is stored in a flattened Bash array.\n5. To compute the transpose, we loop by column index j (from 0 to num_cols–1)\n\u2003 collecting all elements matrix[i][j] (with i = 0 to num_rows–1) into a new row.\n6. Finally, the new rows are combined and printed in the same "[...],[...]" format.\n\nYou can test it like:\n\n\u2003\u2003./transpose.sh "[1,2],[3,4],[5,6]"\n\nwhich should output:\n\n\u2003\u2003[1,3,5],[2,4,6]\n\nFeel free to modify or extend the script as needed for your purposes.',
│   │   │   │   refusal=None,
│   │   │   │   role='assistant',
│   │   │   │   annotations=[],
│   │   │   │   audio=None,
│   │   │   │   function_call=None,
│   │   │   │   tool_calls=None
│   │   │   )
│   │   )
│   ],
│   created=1743700743,
│   model='o3-mini-2025-01-31',
│   object='chat.completion',
│   service_tier='default',
│   system_fingerprint='fp_617f206dd9',
│   usage=CompletionUsage(
│   │   completion_tokens=1806,
│   │   prompt_tokens=44,
│   │   total_tokens=1850,
│   │   completion_tokens_details=CompletionTokensDetails(
│   │   │   accepted_prediction_tokens=0,
│   │   │   audio_tokens=0,
│   │   │   reasoning_tokens=960,
│   │   │   rejected_prediction_tokens=0
│   │   ),
│   │   prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_to

-----------------
Below is one example of a Bash script that reads a matrix in the form
  [1,2],[3,4],[5,6]
and prints its transpose in the same format. Save the script (for example, as transpose.sh),
make it executable (chmod +x transpose.sh), and run it by passing the matrix string as an argument.

#!/bin/bash
# Check if an argument is passed
if [ $# -lt 1 ]; then
  echo "Usage: $0 '[1,2],[3,4],[5,6]'"
  exit 1
fi

# The input matrix string (for example: "[1,2],[3,4],[5,6]")
matrix_string="$1"

# Extract rows using grep so that each row is of the form "[ … ]"
rows=($(echo "$matrix_string" | grep -o '\[[^]]*\]'))
num_rows=${#rows[@]}

if [ $num_rows -eq 0 ]; then
  echo "Error: no rows found. Make sure your input is in the format '[1,2],[3,4],[5,6]'."
  exit 1
fi

declare -a matrix
num_cols=0

# Process each row to extract the individual numbers.
for (( i=0; i<num_rows; i++ )); do
    # Remove the leading [ and trailing ]
    row=$(echo "${rows[$i]}" | sed 's/^\[//; s/\]$//')
    # Sp

# Conversation States, Memory


# TODO

# Agent
## Agent calling tools
1. Simply demonstrates an agent using a tool.
1. Look at the brevity of the code compared to doing a function calling all on our own.
1. Play with the question that can be asked to agent to see how it can handle questions that may or may not require the tool

In [22]:
model = "gpt-4o"
from agents import Agent, ModelSettings, function_tool, Runner

@function_tool
def get_weather(latitude:str, longitude:str) ->str:
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m")
    data = response.json()
    return data['current']['temperature_2m']

#instructions="Always respond in haiku form",    
agent = Agent(
    name="Blaster",
    instructions="Answer very precisely to the question asked. Please think before answering",
    model= model,
    tools=[get_weather],
)

result = await Runner.run(agent, "which is warmer now: Paris or Manila?")
print(result.final_output)
print("--------------------------")
pprint(result)

Manila is warmer than Paris right now.
--------------------------


RunResult(
│   input='which is warmer now: Paris or Manila?',
│   new_items=[
│   │   ToolCallItem(
│   │   │   agent=Agent(
│   │   │   │   name='Blaster',
│   │   │   │   instructions='Answer very precisely to the question asked. Please think before answering',
│   │   │   │   handoff_description=None,
│   │   │   │   handoffs=[],
│   │   │   │   model='gpt-4o',
│   │   │   │   model_settings=ModelSettings(
│   │   │   │   │   temperature=None,
│   │   │   │   │   top_p=None,
│   │   │   │   │   frequency_penalty=None,
│   │   │   │   │   presence_penalty=None,
│   │   │   │   │   tool_choice=None,
│   │   │   │   │   parallel_tool_calls=False,
│   │   │   │   │   truncation=None,
│   │   │   │   │   max_tokens=None
│   │   │   │   ),
│   │   │   │   tools=[
│   │   │   │   │   FunctionTool(
│   │   │   │   │   │   name='get_weather',
│   │   │   │   │   │   description='',
│   │   │   │   │   │   params_json_schema={
│   │   │   │   │   │   │   'properties': {
│   │   │   │   │   │   │   │   'latitude': {'title': 'Latitude', 'type': 'string'},
│   │   │   │   │   │   │   │   'longitude': {'title': 'Longitude', 'type': 'string'}
│   │   │   │   │   │   │   },
│   │   │   │   │   │   │   'required': ['latitude', 'longitude'],
│   │   │   │   │   │   │   'title': 'get_weather_args',
│   │   │   │   │   │   │   'type': 'object',
│   │   │   │   │   │   │   'additionalProperties': False
│   │   │   │   │   │   },
│   │   │   │   │   │   on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x1273f89a0>,
│   │   │   │   │   │   strict_json_schema=True
│   │   │   │   │   )
│   │   │   │   ],
│   │   │   │   mcp_servers=[],
│   │   │   │   input_guardrails=[],
│   │   │   │   output_guardrails=[],
│   │   │   │   output_type=None,
│   │   │   │   hooks=None,
│   │   │   │   tool_use_behavior='run_llm_again',
│   │   │   │   reset_tool_choice=True
│   │   │   ),
│   │   │   raw_item=ResponseFunctionToolCall(
│   │   │   │   arguments='{"latitude":"48.8566","longitude":"2.3522"}',
│   │   │   │   call_id='call_N4sIuCDBf555hzXNV1FIvnJb',
│   │   │   │   name='get_weather',
│   │   │   │   type='function_call',
│   │   │   │   id='fc_67eec318227c819294311156c32fb800097edb372fba9fae',
│   │   │   │   status='completed'
│   │   │   ),
│   │   │   type='tool_call_item'
│   │   ),
│   │   ToolCallOutputItem(
│   │   │   agent=Agent(
│   │   │   │   name='Blaster',
│   │   │   │   instructions='Answer very precisely to the question asked. Please think before answering',
│   │   │   │   handoff_description=None,
│   │   │   │   handoffs=[],
│   │   │   │   model='gpt-4o',
│   │   │   │   model_settings=ModelSettings(
│   │   │   │   │   temperature=None,
│   │   │   │   │   top_p=None,
│   │   │   │   │   frequency_penalty=None,
│   │   │   │   │   presence_penalty=None,
│   │   │   │   │   tool_choice=None,
│   │   │   │   │   parallel_tool_calls=False,
│   │   │   │   │   truncation=None,
│   │   │   │   │   max_tokens=None
│   │   │   │   ),
│   │   │   │   tools=[
│   │   │   │   │   FunctionTool(
│   │   │   │   │   │   name='get_weather',
│   │   │   │   │   │   description='',
│   │   │   │   │   │   params_json_schema={
│   │   │   │   │   │   │   'properties': {
│   │   │   │   │   │   │   │   'latitude': {'title': 'Latitude', 'type': 'string'},
│   │   │   │   │   │   │   │   'longitude': {'title': 'Longitude', 'type': 'string'}
│   │   │   │   │   │   │   },
│   │   │   │   │   │   │   'required': ['latitude', 'longitude'],
│   │   │   │   │   │   │   'title': 'get_weather_args',
│   │   │   │   │   │   │   'type': 'object',
│   │   │   │   │   │   │   'additionalProperties': False
│   │   │   │   │   │   },
│   │   │   │   │   │   on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x1273f89a0>,
│   │   │   │   │   │   strict_json_schema=True
│   │   │   │   │   )
│   │   │   │   ],
│   │   │   │   mcp_servers=[],
│   │   │   │   input_guar

## Agents Collaborating
1. Simply demonstrates an agent reviewing the work of another agent - much like a human being.
1. This is one of the primary reasons while the agents can help increase accuracty of the answer and smaller models using agents can outperform larger models without agents.
1. This pattern can be used in lots of scenarios.

In [23]:
from dataclasses import dataclass
from typing import Literal

from agents import Agent, ItemHelpers, Runner, TResponseInputItem, trace

"""
This example shows the LLM as a judge pattern. The first agent generates an outline for a story.
The second agent judges the outline and provides feedback. We loop until the judge is satisfied
with the outline.
"""
model = "gpt-4o"

story_outline_generator = Agent(
    name="story_outline_generator",
    instructions=(
        "You generate a very short story outline based on the user's input."
        "If there is any feedback provided, use it to improve the outline."
    ),
    model= model,
)


@dataclass
class EvaluationFeedback:
    feedback: str
    score: Literal["pass", "needs_improvement", "fail"]


evaluator = Agent(
    name="evaluator",
    instructions=(
        "You evaluate a story outline and decide if it's good enough."
        "If it's not good enough, you provide feedback on what needs to be improved."
        "Never give it a pass on the first try. Do not spin for more 2 times."
    ),
    model= model,
    output_type=EvaluationFeedback,
)


In [24]:
msg = input("What kind of story would you like to hear? ")
input_items: list[TResponseInputItem] = [{"content": msg, "role": "user"}]

latest_outline: str | None = None

    # We'll run the entire workflow in a single trace
with trace("LLM as a judge"):
        while True:
            story_outline_result = await Runner.run(
                story_outline_generator,
                input_items,
            )

            input_items = story_outline_result.to_input_list()
            latest_outline = ItemHelpers.text_message_outputs(story_outline_result.new_items)
            print("Story outline generated")

            evaluator_result = await Runner.run(evaluator, input_items)
            result: EvaluationFeedback = evaluator_result.final_output

            print(f"Evaluator score: {result.score}")

            if result.score == "pass":
                print("Story outline is good enough, exiting.")
                break

            print("Re-running with feedback")

            input_items.append({"content": f"Feedback: {result.feedback}", "role": "user"})

print(f"Final story outline: {latest_outline}")



What kind of story would you like to hear?  FUNNY


Story outline generated
Evaluator score: needs_improvement
Re-running with feedback
Story outline generated
Evaluator score: needs_improvement
Re-running with feedback
Story outline generated
Evaluator score: pass
Story outline is good enough, exiting.
Final story outline: Thank you! I'm glad you enjoyed the improvements. If you ever want more adventures with Professor Bumble or need further assistance, feel free to reach out. Have fun with your storytelling!


## Agents routing
1. Simply demonstrates an agent routing work to other agents.
1. This is a very common agentic pattern.
1. Ask the question in German and see what happens! In real life when we use a routing pattern, we must have a fallback agent that gracefully handles all things unknown.

In [25]:

import uuid

#from openai.types.responses import ResponseContentPartDoneEvent, ResponseTextDeltaEvent

from agents import Agent, RawResponsesStreamEvent, Runner, TResponseInputItem, trace

"""
This example shows the handoffs/routing pattern. The triage agent receives the first message, and
then hands off to the appropriate agent based on the language of the request. Responses are
streamed to the user.
"""
model = "gpt-4o"

french_agent = Agent(
    name="french_agent",
    instructions="You only speak French",
    model = model,
)

spanish_agent = Agent(
    name="spanish_agent",
    instructions="You only speak Spanish",
    model = model,
)

english_agent = Agent(
    name="english_agent",
    instructions="You only speak English. Answer the question you recieved.",
    model = model,
)

# experiment by removing  from the instructions the sentence below.
#So answer in English even if you understand the language that is being used.
#And then ask say (German) : Wie geht es dir
know_all_agent = Agent(
    name="know_all_agent",
    instructions="You only speak English. So answer in English even if you understand the language that is being used. \
        State that you do not understand the user question and ask them to repeat it one of the languages you understand. \
        Those languages are English, French and Spanish .",
    model = model,
)

triage_agent = Agent(
    name="triage_agent",
    instructions="Handoff to the appropriate agent based on the language of the request. If you do not know what to do, hand if off to know_all.",
    handoffs=[french_agent, spanish_agent, english_agent, know_all_agent],
    model = model,
)



In [26]:

msg = input("Hi! We speak French, Spanish and English. How can I help? ")
inputs: list[TResponseInputItem] = [{"content": msg, "role": "user"}]

with trace("Router"):
        #while True:
    story_outline_result = await Runner.run(triage_agent,inputs)
    print(f"Output Summary: {story_outline_result}")
    print("--------------------------")
    #pprint(story_outline_result)
    print("--------------------------")
    print(story_outline_result.final_output)


Hi! We speak French, Spanish and English. How can I help?  KEMON accho?


Output Summary: RunResult:
- Last agent: Agent(name="know_all_agent", ...)
- Final output (str):
    I’m sorry, I don’t understand. Could you please repeat your question in English, French, or Spanish?
- 3 new item(s)
- 2 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)
--------------------------
--------------------------
I’m sorry, I don’t understand. Could you please repeat your question in English, French, or Spanish?


## Agents Deterministic Workflow
1. Simply demonstrates agents calling other agents to complete a well defined workflow.
1. This is a very common agentic pattern.
1. Ask the question in German and see what happens! In real life when we use a routing pattern, we must have a fallback agent that gracefully handles all things unknown.

# TODO

# AFTERWORD
Agent is an extremely powerful construct in the field of GenAI.....